In [1]:
# Import required libraries
from transformers import BertTokenizer, TFBertForQuestionAnswering
import tensorflow as tf

# Set hyperparameters
max_seq_length = 384
doc_stride = 128
max_query_length = 64
batch_size = 32
learning_rate = 3e-5
num_train_epochs = 2.0

# Load train and test data
train_examples = []
with open('squad_train.tf_record') as f:
  for line in f:
    example = {
        'input_ids': line['input_ids'],
        'attention_mask': line['attention_mask'],
        'token_type_ids': line['token_type_ids'],
        'start_position': line['start_position'],
        'end_position': line['end_position'],
    }
    train_examples.append(example)
num_train_steps = int(len(train_examples) / batch_size * num_train_epochs)
train_dataset = tf.data.Dataset.from_tensor_slices(train_examples)
train_dataset = train_dataset.shuffle(100).batch(batch_size).prefetch(10)
test_examples = []
with open('squad_test.tf_record') as f:
  for line in f:
    example = {
        'input_ids': line['input_ids'],
        'attention_mask': line['attention_mask'],
        'token_type_ids': line['token_type_ids'],
    }
    test_examples.append(example)
test_dataset = tf.data.Dataset.from_tensor_slices(test_examples)
test_dataset = test_dataset.batch(batch_size).prefetch(10)

# Build BERT model
tokenizer = BertTokenizer.from_pretrained('uncased_L-12_H-768_A-12')
model = TFBertForQuestionAnswering.from_pretrained('uncased_L-12_H-768_A-12')
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Fine-tune BERT on train set
for epoch in range(num_train_epochs):
  for step, batch in enumerate(train_dataset):
    with tf.GradientTape() as tape:
      logits = model(batch['input_ids'], attention_mask=batch['attention_mask'], token_type_ids=batch['token_type_ids'])
      start_logits, end_logits = logits
      start_loss = loss(batch['start_position'], start_logits)
      end_loss = loss(batch['end_position'], end_logits)
      total_loss = tf.reduce_mean(start_loss + end_loss)
    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    if step % 100 == 0:
      print('Epoch {} Step {} Loss {:.4f}'.format(epoch + 1, step, total_loss.numpy()))

2022-12-02 13:47:28.654543: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


FileNotFoundError: [Errno 2] No such file or directory: 'squad_train.tf_record'

In [ ]:
# Import required libraries
import json
import random
from rouge import Rouge

# Load SQuAD dataset
with open('train-v1.1.json', 'r') as f:
  squad_dataset = json.load(f)

# Split dataset into train and test sets
random.seed(0)
test_size = 0.1
num_data = len(squad_dataset['data'])
test_data = random.sample(squad_dataset['data'], int(test_size * num_data))
train_data = [d for d in squad_dataset['data'] if d not in test_data]

# Evaluate BERT on ROUGE metrics
rouge = Rouge()
scores = rouge.get_scores(predicted_answers, test_data['answers'])
print(scores)
